In [23]:
(ql:quickload '(:liebler :cytoscape-clj :trivial-do))

To load "liebler":
  Load 1 ASDF system:
    liebler

; Loading "liebler"

To load "cytoscape-clj":
  Load 1 ASDF system:
    cytoscape-clj

; Loading "cytoscape-clj"

To load "trivial-do":
  Load 1 ASDF system:
    trivial-do


(:LIEBLER :CYTOSCAPE-CLJ :TRIVIAL-DO)


; Loading "trivial-do"



In [29]:
(defmethod cytoscape:add-graph (instance graph &rest args)
  (declare (ignore args))
  (let ((id (jupyter:make-uuid)))
    (setf (cytoscape:elements instance)
          (append (cytoscape:elements instance)
                  (liebler:map-vertices 'list
                                        (lambda (vertex)
                                          (make-instance 'cytoscape:element
                                                         :group "nodes"
                                                         :classes (list (format nil "color~3,'0d" (liebler:color graph vertex)))
                                                         :data `(("id" . ,(format nil "~A-~A" id vertex))
                                                                 ("label" . ,(format nil "~A" vertex)))))
                                        graph)
                  (liebler:map-edges 'list
                                     (lambda (vertex1 vertex2)
                                       (make-instance 'cytoscape:element
                                                      :group "edges"
                                                      :data `(("source" . ,(format nil "~A-~A" id vertex1))
                                                              ("target" . ,(format nil "~A-~A" id vertex2)))))
                                     graph))))
  (values))

#<STANDARD-METHOD CYTOSCAPE:ADD-GRAPH (T T) {10061394B3}>

[SB-KERNEL:REDEFINITION-WITH-DEFMETHOD] redefining CYTOSCAPE:ADD-GRAPH (#<SB-PCL:SYSTEM-CLASS COMMON-LISP:T> #<SB-PCL:SYSTEM-CLASS COMMON-LISP:T>) in DEFMETHOD


In [25]:
(defun nshuffle (sequence)
  (loop for i from (length sequence) downto 2
        do (rotatef (elt sequence (random i))
                    (elt sequence (1- i))))
  sequence)

(defun color-styles ()
  (with-output-to-string (stream)
    (trivial-do:dolist* (index color (nshuffle jupyter-widgets::*color-names*))
      (format stream "node.color~3,'0D { background-color: ~A; }~%" index color))))

NSHUFFLE

COLOR-STYLES

[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining COMMON-LISP-USER::NSHUFFLE in DEFUN


In [45]:
(defparameter c (make-instance 'cytoscape:cytoscape-widget
                               :graph-layouts (list (make-instance 'cytoscape:cose-layout))
                               :graph-style (concatenate 'string "node { label: data(label); }" (color-styles))))
c

C

A Jupyter Widget

In [12]:
(defparameter g (liebler:make-adjacency-matrix 6
                                               :edges '((0 . 1) (0 . 4) (1 . 4) (1 . 2)
                                                        (2 . 3) (3 . 4) (3 . 5))))

G

In [46]:
(cytoscape:add-graph c g)

In [47]:
(cytoscape:add-graph c (liebler::colorize (liebler:order-graph-by-color (liebler::color-by-ex-degree g) #'>)))